In [1]:
import pandas as pd
import numpy as np
import sqldf
import requests
import io
import zipfile
import bs4
import re
import string
import matplotlib.pyplot as plt

# tratamento dados

In [2]:
#exemplo de dados
# BPA = balanço patrimonial ativo
# BPP = balanço patrimonial passivo
# DRE = demonstrativo de resultado

class Get_dados():
    def __init__(self, tipo_dado, ano):
        self.tipo_dado = tipo_dado.upper()
        self.ano = ano.upper()
        
    def pegar_dados(self):

        dados = f'dfp_cia_aberta_{self.tipo_dado}_con_{self.ano}.csv'
        link = f'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_{self.ano}.zip'

        r = requests.get(link)

        file = zipfile.ZipFile(io.BytesIO(r.content))
        file = file.open(dados)
        linhas = file.readlines()
        
        linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
        linhas = [i.split(';') for i in linhas]
        
        df = pd.DataFrame(linhas[1:], columns=linhas[0])
        
        return df



def corrigir_cnpj(cnpj):
    cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')

    return cnpj
def pass_numeric(serie):
    serie = pd.to_numeric(serie)
    return serie



In [4]:
#=====================================Tratamento balanço patrimonial (ativo) BPA======================================
df_ativo = Get_dados('bpa','2021').pegar_dados()
df_ativo.CNPJ_CIA = corrigir_cnpj(df_ativo.CNPJ_CIA)
df_ativo.VL_CONTA = pass_numeric(df_ativo.VL_CONTA)

#=====================================Tratamento balanço patrimonial (passivo) BPP===================================
df_passivo = Get_dados('bpp','2021').pegar_dados()
df_passivo.CNPJ_CIA = corrigir_cnpj(df_passivo.CNPJ_CIA)
df_passivo.VL_CONTA = pass_numeric(df_passivo.VL_CONTA)

#=====================================Tratamento para o DRE==========================================================
df_dre = Get_dados('dre','2021').pegar_dados()
df_dre.CNPJ_CIA = corrigir_cnpj(df_dre.CNPJ_CIA)
df_dre.VL_CONTA = pass_numeric(df_dre.VL_CONTA)

#====================================Concatenacao ativo+passivo=====================================================
new_df = pd.concat([df_ativo,df_passivo])

#====================================Query para bp==================================================================
query_bp = """

select *
from new_df
group by new_df.CNPJ_CIA, new_df.ORDEM_EXERC, new_df.DS_CONTA
order by new_df.CNPJ_CIA, new_df.CD_CONTA
"""
new_bp = sqldf.run(query_bp)
new_bp = new_bp.loc[new_bp.ORDEM_EXERC == 'ÚLTIMO']

new_bp = new_bp.loc[(new_bp['DENOM_CIA'] != 'BRB BCO DE BRASILIA S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB')&
                   (new_bp['DENOM_CIA'] != 'BANCO INTER S.A.')&
                   (new_bp['DENOM_CIA'] != 'PORTO SEGURO S.A.')&
                   (new_bp['DENOM_CIA'] != 'BRAZILIAN FINANCE E REAL ESTATE S.A.')&
                   (new_bp['DENOM_CIA'] != 'FINANCEIRA ALFA S.A.- CRED FINANC E INVS')&
                   (new_bp['DENOM_CIA'] != 'BCO MERCANTIL DO BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'NU HOLDINGS LTD.')&
                   (new_bp['DENOM_CIA'] != 'BANESTES S.A. - BCO EST ESPIRITO SANTO')&
                   (new_bp['DENOM_CIA'] != 'BCO ABC BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BANCO MODAL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO PAN S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO BRADESCO S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO ALFA DE INVESTIMENTO S.A.')&
                   (new_bp['DENOM_CIA'] != 'ITAU UNIBANCO HOLDING S.A.')&
                   (new_bp['DENOM_CIA'] != 'BANCO BMG S/A')&
                   (new_bp['DENOM_CIA'] != 'BCO PINE S.A')&
                   (new_bp['DENOM_CIA'] != 'BCO DAYCOVAL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BANCO RCI BRASIL S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO SANTANDER (BRASIL) S.A.')&
                   (new_bp['DENOM_CIA'] != 'BCO ESTADO DO RIO GRANDE DO SUL S.A.')
                  ]




#====================================Query para dre=====================================================

query_dre = """
select *
from df_dre
group by df_dre.CNPJ_CIA, df_dre.ORDEM_EXERC, df_dre.DS_CONTA
order by df_dre.CNPJ_CIA ,df_dre.CD_CONTA

"""

new_dre = sqldf.run(query_dre)
new_dre = new_dre.loc[new_dre.ORDEM_EXERC == 'ÚLTIMO']


new_dre = new_dre.loc[(new_dre['DENOM_CIA'] != 'BRB BCO DE BRASILIA S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB')&
                   (new_dre['DENOM_CIA'] != 'BANCO INTER S.A.')&
                   (new_dre['DENOM_CIA'] != 'PORTO SEGURO S.A.')&
                   (new_dre['DENOM_CIA'] != 'BRAZILIAN FINANCE E REAL ESTATE S.A.')&
                   (new_dre['DENOM_CIA'] != 'FINANCEIRA ALFA S.A.- CRED FINANC E INVS')&
                   (new_dre['DENOM_CIA'] != 'BCO MERCANTIL DO BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'NU HOLDINGS LTD.')&
                   (new_dre['DENOM_CIA'] != 'BANESTES S.A. - BCO EST ESPIRITO SANTO')&
                   (new_dre['DENOM_CIA'] != 'BCO ABC BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BANCO MODAL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO PAN S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO BRADESCO S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO ALFA DE INVESTIMENTO S.A.')&
                   (new_dre['DENOM_CIA'] != 'ITAU UNIBANCO HOLDING S.A.')&
                   (new_dre['DENOM_CIA'] != 'BANCO BMG S/A')&
                   (new_dre['DENOM_CIA'] != 'BCO PINE S.A')&
                   (new_dre['DENOM_CIA'] != 'BCO DAYCOVAL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BANCO RCI BRASIL S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO SANTANDER (BRASIL) S.A.')&
                   (new_dre['DENOM_CIA'] != 'BCO ESTADO DO RIO GRANDE DO SUL S.A.')
                  ]



#===================================ÍNDICES=====================================
#=====dicionário
# 3.07 ---> Resultado Antes dos Tributos sobre o Lucro (EBIT)
# 3.01 ---> Receita de Venda de Bens e/ou Serviços
# 2.03 ---> Patrmônio Líquido
# 1 ---> ativo total
# 1.02.03 ---> imobilizado
# 1.01.08.03.01 --> financiamento e empréstimos
# 1.01 ---> ativo circulante
# 2.01 ---> passivo circulante
# 3.11 ---> lucro/prejuizo consolidado do período (rol)
# 2.01.04 ---> empréstimos e financiamentos (DOL)>dívida liquida onerosa


indices_ebit  = list(map(list,np.where(new_dre.CD_CONTA == '3.07')))[0]
indices_receita_total = list(map(list,np.where(new_dre.CD_CONTA == '3.01')))[0]
indices_patrimonio_liquido = list(map(list,np.where(new_bp.CD_CONTA == '2.03')))[0]
indices_ativo_total = list(map(list,np.where(new_bp.CD_CONTA == '1')))[0]
indices_imobilizado = list(map(list,np.where(new_bp.CD_CONTA == '1.02.03')))[0]
indices_ativo_circulante = list(map(list,np.where(new_bp.CD_CONTA == '1.01')))[0]
indices_passivo_circulante = list(map(list,np.where(new_bp.CD_CONTA == '2.01')))[0]
indices_emprestimos_financiamento = list(map(list,np.where(new_bp.CD_CONTA == '2.01.04')))[0]
indices_lucro_periodo = list(map(list,np.where(new_dre.CD_CONTA == '3.11')))[0]
indices_caixa = list(map(list,np.where(new_bp.CD_CONTA == '1.01.01')))[0]

empresas = []
ebit = []
receita_total = []
pl = []
ativo_total = []
imobilizado = []
ativo_circulante = []
passivo_circulante = []
lucro_periodo = []
emprestimos_financiamento = []
cnpj = []


#====================lucro do periodo====================
for i in indices_lucro_periodo:
    lucro_periodo.append(new_dre.iloc[i,14])

#====================indices_emprestimos_financiamento====================
for i in indices_emprestimos_financiamento:
    emprestimos_financiamento.append(new_bp.iloc[i,13])
    
#====================EBIT====================
for i in indices_ebit:
    empresas.append(new_dre.iloc[i,4])
    cnpj.append(new_dre.iloc[i,1])
    ebit.append(new_dre.iloc[i,14])

#======================receita total======================
for i in indices_receita_total:
    receita_total.append(new_dre.iloc[i,14])

#=====================Patrimônio líquido=================
for i in indices_patrimonio_liquido:
    pl.append(new_bp.iloc[i,13])

#=====================Ativo total=================
for i in indices_ativo_total:
    ativo_total.append(new_bp.iloc[i,13])
#=====================Imobilizado=================
for i in indices_imobilizado:
    imobilizado.append(new_bp.iloc[i,13])

#=====================ativo circulante=================
for i in indices_ativo_circulante:
    ativo_circulante.append(new_bp.iloc[i,13])

#=====================passivo circulante=================
for i in indices_passivo_circulante:
    passivo_circulante.append(new_bp.iloc[i,13])

    
    
    
#=============================construção df=========================    

df = pd.DataFrame({"cnpj":cnpj,
                    "empresas":empresas,
                   "ativo_total":ativo_total,
                   "ativo_circulante":ativo_circulante,
                   "passivo_circulante":passivo_circulante,
                   "ebit":ebit,
                   "receita_total":receita_total,
                   "capitais_de_terceiros":emprestimos_financiamento,
                   "patrimonio_liquido":pl,
                    "imobilizado":imobilizado,
                    "lucro_periodo":lucro_periodo})


#==================indicadores estrutura de capital=====================
df['imobilizacao_do_pl'] = round(df.imobilizado/df.patrimonio_liquido*100,2)
df['composicao_endividamento'] = round(df.passivo_circulante/df.capitais_de_terceiros*100,2)
df['participacao_capitais_de_terceiros'] = round(df.capitais_de_terceiros/df.patrimonio_liquido*100,2)
#=======================================================================================================

#==================indicador de liquidez=========================================
df['liquidez_corrente'] = round(df.ativo_circulante/df.passivo_circulante,2)

#==============================indicador de rentabilidade==================================================
df['rentabilidade_ativo'] = round(df.lucro_periodo/df.ativo_total*100,2)

#==========================================================================================================

df['porte_rol'] = round(np.log(df.lucro_periodo*1000),2)
df['porte_ativo'] = round(np.log(df.ativo_total*1000),2)

#df = df.sort_values(by=['ebit/receita_total'], ascending=False)

query_geset = """

select 
df.empresas,
df.imobilizacao_do_pl,
df.composicao_endividamento,
df.participacao_capitais_de_terceiros,
df.liquidez_corrente,
df.rentabilidade_ativo,
df.porte_rol,
df.porte_ativo

from df

"""
risk = sqldf.run(query_geset)


/tmp/ipykernel_4231/2979979761.py:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cnpj = cnpj.str.replace('.','').str.replace('/','').str.replace('-','')
/home/felipe/jupyter_dir/jupyter_env/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
#==================tratamento estatístico=========================
risk = risk.loc[(risk['imobilizacao_do_pl'] != 'inf')&
                (risk['imobilizacao_do_pl'] != 0)&
                
                   (risk['composicao_endividamento'] != 'inf')&
                   (risk['composicao_endividamento'] != 0)&
                
                   (risk['participacao_capitais_de_terceiros'] != 'inf')&
                   (risk['participacao_capitais_de_terceiros'] != 0)&
                
                   (risk['liquidez_corrente'] != 'inf')&
                   (risk['liquidez_corrente'] != 0)&
                
                   (risk['rentabilidade_ativo'] != 'inf')&
                   (risk['rentabilidade_ativo'] != 0)&
                
                   (risk['porte_rol'] != 'inf')&
                   (risk['porte_rol'] != 0)&
                
                
                   (risk['porte_ativo'] != 'inf')&
                   (risk['porte_ativo'] != 0)]

# risk = risk.set_index(['empresas'])
risk = risk.dropna()

df = df.loc[(df['imobilizacao_do_pl'] != 'inf')&
                (df['imobilizacao_do_pl'] != 0)&
                
                   (df['composicao_endividamento'] != 'inf')&
                   (df['composicao_endividamento'] != 0)&
                
                   (df['participacao_capitais_de_terceiros'] != 'inf')&
                   (df['participacao_capitais_de_terceiros'] != 0)&
                
                   (df['liquidez_corrente'] != 'inf')&
                   (df['liquidez_corrente'] != 0)&
                
                   (df['rentabilidade_ativo'] != 'inf')&
                   (df['rentabilidade_ativo'] != 0)&
                
                   (df['porte_rol'] != 'inf')&
                   (df['porte_rol'] != 0)&
                
                
                   (df['porte_ativo'] != 'inf')&
                   (df['porte_ativo'] != 0)]

df = df.dropna()
                   

In [185]:
# sorted(iterável, chave=Nenhum, reverse=Falso)

In [7]:
im_pl = sorted(risk.imobilizacao_do_pl,reverse=True)
cmp_end = sorted(risk.composicao_endividamento,reverse=True)
prt_cpt_terc = sorted(risk.participacao_capitais_de_terceiros,reverse=True)
liq_corr = sorted(risk.liquidez_corrente)
rent_ativo = sorted(risk.rentabilidade_ativo)
port_rol = sorted(risk.porte_rol)
port_ativo = sorted(risk.porte_ativo)


percent_im_pl = []
percent_cmp_end = []
percent_prt_cpt_terc = []
percent_liq_corr = []
percent_rent_ativo = []
percent_port_rol = []
percent_port_ativo = []

percents = [10.0, 20.0, 30.0, 40.0, 50.0,  60.0,  70.0, 80.0, 90.0]

for i in percents:
    percent_im_pl.append(np.percentile(im_pl,i))
    percent_cmp_end.append(np.percentile(cmp_end,i))
    percent_prt_cpt_terc.append(np.percentile(prt_cpt_terc,i))
    percent_liq_corr.append(np.percentile(liq_corr,i))
    percent_rent_ativo.append(np.percentile(rent_ativo,i))
    percent_port_rol.append(np.percentile(port_rol,i))
    percent_port_ativo.append(np.percentile(port_ativo,i))
    
def decil():    
    df_decil= pd.DataFrame({"imbolizacao_pl":sorted(percent_im_pl,reverse=True),
                               "composicao_endividamente":sorted(percent_cmp_end,reverse=True),
                               "participacao_capital_de_terceiros":sorted(percent_prt_cpt_terc,reverse=True),
                               "liquidez_corrente":percent_liq_corr,
                               "rentabilidade_ativo":percent_rent_ativo,
                               "porte_rol":percent_port_rol,
                               "porte_ativo":percent_port_ativo})
    return df_decil

def indicadores():

    df_ind = df_percents = pd.DataFrame({"imbolizacao_pl":im_pl,
                               "composicao_endividamente":cmp_end,
                               "participacao_capital_de_terceiros":prt_cpt_terc,
                               "liquidez_corrente":liq_corr,
                               "rentabilidade_ativo":rent_ativo,
                               "porte_rol":port_rol,
                               "porte_ativo":port_ativo})
    return df_ind

In [11]:
df.head(50)

,cnpj,empresas,ativo_total,ativo_circulante,passivo_circulante,ebit,receita_total,capitais_de_terceiros,patrimonio_liquido,imobilizado,lucro_periodo,imobilizacao_do_pl,composicao_endividamento,participacao_capitais_de_terceiros,liquidez_corrente,rentabilidade_ativo,porte_rol,porte_ativo
0,00001180000126,CENTRAIS ELET BRAS S.A. - ELETROBRAS,188303069.0,39744996.0,23883372.0,10994356.0,37616241.0,8444527.0,76416764.0,33367981.0,5713633.0,43.67,282.83,11.05,1.66,3.03,22.47,25.96
1,00080671000100,CARAMURU ALIMENTOS S.A.,6550203.0,4902276.0,3597920.0,369879.0,7592251.0,1926857.0,1514743.0,948645.0,353867.0,62.63,186.72,127.21,1.36,5.40,19.68,22.60
2,00194724000113,AES BRASIL OPERAÇÕES S.A.,11768649.0,2279670.0,1156862.0,-30499.0,2511746.0,286266.0,4613565.0,7343248.0,516512.0,159.17,404.12,6.20,1.97,4.39,20.06,23.19
3,00242184000104,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",3281487.0,1897148.0,311413.0,89589.0,389754.0,170624.0,1112697.0,1251287.0,58064.0,112.46,182.51,15.33,6.09,1.77,17.88,21.91
5,00389481000179,LM TRANSPORTES INTERESTADUAIS SERVIÇOS E COMÉR...,1977135.0,436440.0,656585.0,150240.0,786738.0,464497.0,269913.0,1492605.0,98665.0,552.99,141.35,172.09,0.66,4.99,18.41,21.40
6,00622416000141,STATKRAFT ENERGIAS RENOVAVEIS S.A.,3431917.0,293508.0,407993.0,211596.0,565054.0,244953.0,2067838.0,2554763.0,183619.0,123.55,166.56,11.85,0.72,5.35,19.03,21.96
7,00776574000156,AMERICANAS S.A.,44518812.0,21912888.0,13336854.0,-16236.0,22695861.0,1052914.0,15910976.0,7825191.0,543795.0,49.18,1266.66,6.62,1.64,1.22,20.11,24.52
8,00864214000106,ENERGISA S.A.,54768404.0,15527778.0,12716103.0,3851503.0,26797969.0,3878428.0,9531241.0,717518.0,3068833.0,7.53,327.87,40.69,1.22,5.60,21.84,24.73
10,01083200000118,NEOENERGIA S.A.,85800000.0,21780000.0,19940000.0,5589000.0,43165000.0,7940000.0,24238000.0,9560000.0,4066000.0,39.44,251.13,32.76,1.09,4.74,22.13,25.18
11,01107327000120,BBM LOGÍSTICA S.A.,962193.0,388307.0,429248.0,-26280.0,1356199.0,149110.0,136704.0,323038.0,12023.0,236.30,287.87,109.08,0.90,1.25,16.30,20.68
